In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
#!pip install python_highcharts
from highcharts import Highchart
import json

Using TensorFlow backend.


In [2]:
def ProcessSilentData(x_train, x_test):
    def proc(x_train_silent):
        length = len(x_train_silent)
        temp = np.zeros((length, 10,46))
        for i in range(len(x_train_silent)):
            for j in range(10):
                for k in range(190):
                    if 146 <= k <= 185:
                        temp[i][j][k-146] = x_train_silent[i][j][k]
                temp[i][j][40] = x_train_silent[i][j][67]#face
                temp[i][j][41] = x_train_silent[i][j][68]
                temp[i][j][42] = x_train_silent[i][j][8]#left hand
                temp[i][j][43] = x_train_silent[i][j][9]
                temp[i][j][44] = x_train_silent[i][j][14]#right hand
                temp[i][j][45] = x_train_silent[i][j][15]
                
        return temp
    x_train_40 = proc(x_train)
    x_test_40 = proc(x_test)
    x_train = x_train_40
    x_test = x_test_40

    #define distance between two points
    def calDis(material, a, b):
        #material = x_train[0][0]
        ax = (a-1) *2
        ay = ax + 1
        bx = (b-1) *2
        by = bx + 1
        d1 = abs(material[ax] - material[bx])
        d2 = abs(material[ay] - material[by])
        dis = np.sqrt(d1*d1 + d2*d2)
        return dis

    def getSilent(x):#x_train[0][0]
        mouthLen = calDis(x, 13, 17)
        sideLen = calDis(x, 1, 12) + calDis(x, 1, 2) + calDis(x, 8, 7) + calDis(x, 6, 7)
        mouthWid = calDis(x, 14, 20) + calDis(x, 15, 19) + calDis(x, 16, 18)
        handLen = calDis(x, 21, 22) + calDis(x, 21, 23)
        
        
        if mouthLen == 0:
            silentWeight = 0
        else:
            silentWeight = mouthWid / mouthLen
        if sideLen == 0:
            sideWeight = 0
            handWeight = 0
        else:
            sideWeight = mouthWid / sideLen
            handWeight = handLen / sideLen
            
        

        return [sideWeight,silentWeight]

    def proX(x_train):
        pro_x_train = np.zeros((len(x_train), 10, 4))
        for i in range(len(x_train)):
            for j in range(len(x_train[i])):
                var0, var1 = getSilent(x_train[i][j])
                pro_x_train[i][j][0] = var0
                pro_x_train[i][j][1] = var1
        
        
        
        for i in range(len(pro_x_train)):#10 2
            varList1 = []
            varList2 = []
            for j in range(10):
                varList1.append(pro_x_train[i][j][0])
                varList2.append(pro_x_train[i][j][1])
            var2 = np.var(varList1)
            var3 = np.var(varList2)
            for j in range(10):

                pro_x_train[i][j][2] = var2
                pro_x_train[i][j][3] = var3
            
            
                
        
        return pro_x_train
    return proX(x_train), proX(x_test)

In [3]:
def ProcessEatData(x_train, x_test):
    def proc(x_train_silent):
        length = len(x_train_silent)
        temp = np.zeros((length, 10,56))
        for i in range(len(x_train_silent)):
            for j in range(10):
                for k in range(190):
                    if 146 <= k <= 185:
                        temp[i][j][k-146] = x_train_silent[i][j][k]
                temp[i][j][40] = x_train_silent[i][j][0]#
                temp[i][j][41] = x_train_silent[i][j][1]
                temp[i][j][42] = x_train_silent[i][j][2]#
                temp[i][j][43] = x_train_silent[i][j][3]
                temp[i][j][44] = x_train_silent[i][j][4]#
                temp[i][j][45] = x_train_silent[i][j][5]
                
                temp[i][j][46] = x_train_silent[i][j][6]#
                temp[i][j][47] = x_train_silent[i][j][7]
                temp[i][j][48] = x_train_silent[i][j][8]#
                temp[i][j][49] = x_train_silent[i][j][9]
                temp[i][j][50] = x_train_silent[i][j][10]#
                temp[i][j][51] = x_train_silent[i][j][11]
                
                temp[i][j][52] = x_train_silent[i][j][12]#
                temp[i][j][53] = x_train_silent[i][j][13]
                temp[i][j][54] = x_train_silent[i][j][14]#
                temp[i][j][55] = x_train_silent[i][j][15]

                
        return temp
    x_train_40 = proc(x_train)
    x_test_40 = proc(x_test)
    x_train = x_train_40
    x_test = x_test_40

    #define distance between two points
    def calDis(material, a, b):
        #material = x_train[0][0]
        ax = (a-1) *2
        ay = ax + 1
        bx = (b-1) *2
        by = bx + 1
        d1 = abs(material[ax] - material[bx])
        d2 = abs(material[ay] - material[by])
        dis = np.sqrt(d1*d1 + d2*d2)
        return dis

    def getSilent(x):#x_train[0][0]
        mouthLen = calDis(x, 13, 17)
        sideLen = calDis(x, 1, 12) + calDis(x, 1, 2) + calDis(x, 8, 7) + calDis(x, 6, 7)
        mouthWid = calDis(x, 14, 20) + calDis(x, 15, 19) + calDis(x, 16, 18)
        handLen = calDis(x, 21, 22) + calDis(x, 21, 23)
        
        
        if mouthLen == 0:
            silentWeight = 0
        else:
            silentWeight = mouthWid / mouthLen
        if sideLen == 0:
            sideWeight = 0
            handWeight = 0
        else:
            sideWeight = mouthWid / sideLen
            handWeight = handLen / sideLen
            
        

        return [sideWeight,silentWeight, x[40],x[41],x[42],x[43],x[44],x[45],x[46],x[47],x[48],x[49],x[50],x[51],x[52],x[53],x[54],x[55]]

    def proX(x_train):
        pro_x_train = np.zeros((len(x_train), 10, 18))
        for i in range(len(x_train)):
            for j in range(len(x_train[i])):
                pro_x_train[i][j] = getSilent(x_train[i][j])
        return pro_x_train


    return proX(x_train), proX(x_test)

In [4]:
x_train = np.load("10/model_data/balancetalking/x_train_talking.npy")
x_test = np.load("10/model_data/balancetalking/x_test_talking.npy")
x_train, x_test = ProcessSilentData(x_train, x_test)###reduced
y_train = np.load("10/model_data/balancetalking/y_train_talking.npy")
y_test = np.load("10/model_data/balancetalking/y_test_talking.npy")


In [5]:
model = load_model('10/silverSilent.h5')
r7 = model.predict(x_train)
r8 = model.predict(x_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
x_train = np.load("10/model_data/balancetalking/x_train_talking.npy")
x_test = np.load("10/model_data/balancetalking/x_test_talking.npy")
x_train, x_test = ProcessSilentData(x_train, x_test)###reduced
y_train = np.load("10/model_data/balancetalking/y_train_talking.npy")
y_test = np.load("10/model_data/balancetalking/y_test_talking.npy")

In [7]:
model = load_model('10/selfeating.h5')
r3 = model.predict(x_train)
r4 = model.predict(x_test)

In [8]:
X_TRAIN = []
for i in range(len(r3)):
    X_TRAIN.append([[r3[i][0]],[r7[i][0]]])
X_TRAIN = np.array(X_TRAIN)
print(X_TRAIN.shape)

(18688, 2, 1)


In [9]:
X_TEST = []
for i in range(len(r4)):
    X_TEST.append([[r4[i][0]],[r8[i][0]]])
X_TEST = np.array(X_TEST)
print(X_TEST.shape)

(8013, 2, 1)


In [10]:
np.save("10/model_data/ens/bronze_X_TEST.npy",X_TEST)
np.save("10/model_data/ens/bronze_X_TRAIN.npy",X_TRAIN)
np.save("10/model_data/ens/bronze_Y_TEST.npy",y_test)
np.save("10/model_data/ens/bronze_Y_TRAIN.npy",y_train)